In [1]:
%env PYTHON_LOGLEVEL=DEBUG

env: PYTHON_LOGLEVEL=DEBUG


In [8]:
from logging import getLogger, StreamHandler, WARNING
import os

PYTHON_LOGLEVEL = os.environ.get("PYTHON_LOGLEVEL", WARNING)
logger = getLogger(__name__)
if not logger.hasHandlers():
    handler = StreamHandler()
    handler.setLevel(PYTHON_LOGLEVEL)
    logger.setLevel(PYTHON_LOGLEVEL)
    logger.addHandler(handler)
    logger.propagate = False

In [23]:
from collections import defaultdict
from math import factorial
from unittest.mock import MagicMock


try:
    logger # type: ignore
except NameError:
    logger = MagicMock()


def avoid_k_palindrome_2() -> int:
    n, k = map(int, input().split())
    s = input()

    # 以降、回文の片側を翼と呼ぶ
    翼長 = k //2

    辞書: defaultdict[str, int] = defaultdict(lambda: 0)
    for char in s:
        辞書[char] += 1
    翼候補 = {key: value for key, value in 辞書.items() if 2 <=value}
    一般文字 = {key: value for key, value in 辞書.items() if value <= 1}
    logger.debug(f"{翼候補=}, {一般文字=}")

    回文の有無を問わない全通り = factorial(n)
    for value in 辞書.values():
        回文の有無を問わない全通り //= factorial(value)
    logger.debug(f"{回文の有無を問わない全通り=}")

    # 回文が作れない場合は全通りを返す
    if sum(翼候補.values()) // 2 < 翼長:
        logger.debug(f"{sum(翼候補.values()) // 2 < 翼長=}, {回文の有無を問わない全通り=}")
        return 回文の有無を問わない全通り

    # 回文が作れる場合、文字列の全通りから、回文を含む文字数全通りを引く
    def 与えられた文字列を翼の部分として含む文字列のパターン数(選ばれた翼候補: str, 残っている翼候補: dict[str, int]) -> int:
        logger.debug(f"{選ばれた翼候補=}, {残っている翼候補=}")

        定員 = len(選ばれた翼候補) == 翼長
        if 定員:
            # ある回文を含む文字列の全通りは、その回文を1文字と見なした文字列の全通りに等しい
            # 回文が奇数文字の場合でも、全通りの数を数えるだけなら、偶数文字の場合と同様に数えられる
            選んだ回文を含む文字列の全通り = 1
            回文の両翼を合わせて1文字と見なした文字数 = len(一般文字.keys()) + sum(残っている翼候補.values()) + 1
            for i in range(1, 回文の両翼を合わせて1文字と見なした文字数 + 1):
                選んだ回文を含む文字列の全通り *= i
            # 組合せなので重複する文字数で割る。
            for value in 残っている翼候補.values():
                選んだ回文を含む文字列の全通り //= value

            # このアプローチは失敗する！！！"yzyzx"のように、1文に2つ以上の回文が含まれる可能性があるため。
            logger.debug(f"{定員=}, {選ばれた翼候補=}, {残っている翼候補=}, {選んだ回文を含む文字列の全通り=}, {回文の両翼を合わせて1文字と見なした文字数=}")
            return 選んだ回文を含む文字列の全通り
        
        else:
            選んだ回文を含む文字列の全通り = 0
            for 次に選ぶ翼候補 in 残っている翼候補.keys():
                次も残っている翼候補 = {
                    key: value for key, value in 残っている翼候補.items() if key != 次に選ぶ翼候補
                }
                次に選ぶ翼候補の残り文字数 = 残っている翼候補[次に選ぶ翼候補] - 2
                if 2 <= 次に選ぶ翼候補の残り文字数:
                    次も残っている翼候補[次に選ぶ翼候補] = 次に選ぶ翼候補の残り文字数
                選んだ回文を含む文字列の全通り += 与えられた文字列を翼の部分として含む文字列のパターン数(選ばれた翼候補 + 次に選ぶ翼候補, 次も残っている翼候補)
            
            logger.debug(f"{定員=}, {選ばれた翼候補=}, {残っている翼候補=}, {選んだ回文を含む文字列の全通り=}")
            return 選んだ回文を含む文字列の全通り

    回文を含む文字列の全通り = 与えられた文字列を翼の部分として含む文字列のパターン数("", 翼候補)
    return 回文の有無を問わない全通り - 回文を含む文字列の全通り

In [24]:
from unittest.mock import patch

with patch(
    "builtins.input",
    side_effect="""
5 3
zzyyx
""".strip().splitlines(),
):
    expected = 16
    actual = avoid_k_palindrome_2()
    print(actual)
    assert expected == actual, print(f"{expected=}, {actual=}")

翼候補={'z': 2, 'y': 2}, 一般文字={'x': 1}
回文の有無を問わない全通り=30
選ばれた翼候補='', 残っている翼候補={'z': 2, 'y': 2}
選ばれた翼候補='z', 残っている翼候補={'y': 2}
定員=True, 選ばれた翼候補='z', 残っている翼候補={'y': 2}, 選んだ回文を含む文字列の全通り=12, 回文の両翼を合わせて1文字と見なした文字数=4
選ばれた翼候補='y', 残っている翼候補={'z': 2}
定員=True, 選ばれた翼候補='y', 残っている翼候補={'z': 2}, 選んだ回文を含む文字列の全通り=12, 回文の両翼を合わせて1文字と見なした文字数=4
定員=False, 選ばれた翼候補='', 残っている翼候補={'z': 2, 'y': 2}, 選んだ回文を含む文字列の全通り=24


6
expected=16, actual=6


AssertionError: None